In [1]:
'''READ ME: 

This is a Python Jupyter Notebook designed to convert any KMZ into a DGN with the proper spatial reference.
Please enter the necessary user inputs below and run the script as needed.

Best practice as always to bring all of the files locally somehwere on the C: Drive for the best performance.

Sebastian Brauer - 2024/07/31
'''

import arcpy
from datetime import datetime
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

input_project_name = 'CSAH_16_ADA' # REQUIRED USER INPUT come up with a name for the project. The goal is to come up with a unique name, the city and TH make a good combo, feel free to add your last name as well. 
input_county_name = 'Washington'# REQUIRED USER INPUT in order to get the right projection. Please capitalize the first letter of the county, feel free to review the dictionary if needed.
input_kml_path = r'I:\Library\Isthmus\GIS\Soft Areas.kmz'# REQUIRED USER INPUT in order to find the file needed for the conversion
input_save_location = r'C:\Users\sbrauer\OneDrive - Isthmus Engineering\Documents\2024' # REQUIRED USER INPUT save location of the output csv should be a folder
input_polygon_indicator = 'N' # REQUIRED USER INPUT if you're kml file has polygons set to 'Y' at least one of the two indicators must be 'Y'
input_polyline_indicator = 'Y' # REQUIRED USER INPUT if you're kml file has polylines set to 'Y' at least one of the two indicators must be 'Y'

# Here is a dictionary that relates every county to a spatial projection number.

mn_county_systems_epsg = {
    "Aitkin": 103700,
    "Anoka": 103708,
    "Becker": 103709,
    "Beltrami North": 103710,
    "Beltrami South": 103711,
    "Benton": 103712,
    "Big Stone": 103713,
    "Blue Earth": 103714,
    "Brown": 103715,
    "Carlton": 103716,
    "Carver": 103717,
    "Cass North": 103718,
    "Cass South": 103719,
    "Chippewa": 103720,
    "Chisago": 103721,
    "Clay": 103701,
    "Clearwater": 103702,
    "Cook North": 103722,
    "Cook South": 103723,
    "Cottonwood": 103724,
    "Crow Wing": 103725,
    "Dakota": 103726,
    "Dodge": 103727,
    "Douglas": 103728,
    "Faribault": 103729,
    "Fillmore": 103730,
    "Freeborn": 103731,
    "Goodhue": 103732,
    "Grant": 103733,
    "Hennepin": 103734,
    "Houston": 1037235,
    "Hubbard": 103703,
    "Isanti": 103736,
    "Itasca North": 103737,
    "Itasca South": 103738,
    "Jackson": 103739,
    "Kanabec": 103740,
    "Kandiyohi": 103741,
    "Kittson": 103742,
    "Koochiching": 103743,
    "Lac qui Parle": 103744,
    "Lake": 103704,
    "Lake of the Woods North": 103745,
    "Lake of the Woods South": 103746,
    "Le Sueur": 103747,
    "Lincoln": 103748,
    "Lyon": 103749,
    "Mahnomen": 103751,
    "Marshall": 103752,
    "Martin": 103753,
    "McLeod": 103750,
    "Meeker": 103754,
    "Mille Lacs": 103705,
    "Morrison": 103755,
    "Mower": 103756,
    "Murray": 103757,
    "Nicollet": 103758,
    "Nobles": 103759,
    "Norman": 103760,
    "Olmsted": 103761,
    "Ottertail": 103762,
    "Pennington": 103763,
    "Pine": 103764,
    "Pipestone": 103765,
    "Polk": 103766,
    "Pope": 103767,
    "Ramsey": 103768,
    "Red Lake": 103769,
    "Redwood": 103770,
    "Renville": 103771,
    "Rice": 103772,
    "Rock": 103773,
    "Roseau": 103774,
    "St. Louis": 103695,
    "St. Louis Central": 103776,
    "St. Louis North": 103775,
    "St. Louis South": 103777,
    "Scott": 103778,
    "Sherburne": 103779,
    "Sibley": 103780,
    "Stearns": 103781,
    "Steele": 103782,
    "Stevens": 103783,
    "Swift": 103784,
    "Todd": 103785,
    "Traverse": 103786,
    "Wabasha": 103787,
    "Wadena": 103788,
    "Waseca": 103789,
    "Washington": 103706,
    "Watonwan": 103790,
    "Wilkin": 103707,
    "Winona": 103791,
    "Wright": 103792,
    "Yellow Medicine": 103793
}

epsg_code = mn_county_systems_epsg.get(input_county_name) # Use the input county name to get the WKID/ESPG
spatial_reference = arcpy.SpatialReference(epsg_code) # Create a Spatial Reference item with this projection WKID/ESPG


output_folder = r'C:\GIS_Scripting\Upload Project\KML Files' # Location where 
arcpy.conversion.KMLToLayer(input_kml_path, output_folder, input_project_name) # Run the KML Conversion Tool


arcpy.env.workspace = fr'C:\GIS_Scripting\Upload Project\KML Files\{input_project_name}.gdb' # DO NOT CHANGE
arcpy.env.overwriteOutput = True

def project_layer(input_layer):
    input_data = fr'Placemarks\{input_layer}'
    output_layer = f'{input_layer}_Projected'
    arcpy.management.Project(input_data, output_layer, spatial_reference)
    return output_layer


if input_polyline_indicator == 'Y' and input_polygon_indicator == 'N': # This whole block will only run if there are polylines
    output_layer = project_layer("Polylines")

elif input_polygon_indicator == 'Y' and input_polyline_indicator == 'N': # This whole block will only run if there are polygons
    polyline_layer = output_layer
    output_layer = project_layer("Polygons")
    output_layer = f'{polyline_layer};{output_layer}'
else:
    polyline_layer = project_layer("Polylines")
    output_layer = project_layer("Polygons")
    output_layer = f'{polyline_layer};{output_layer}'
    

arcpy.conversion.ExportCAD( # Run the export to CAD tool
in_features = output_layer,
Output_Type = "DGN_V8",
Output_File = f'{input_save_location}\{input_project_name}_kmlconversion.dgn',
Ignore_FileNames = "Ignore_Filenames_in_Tables",
Append_To_Existing = "Overwrite_Existing_Files",
Seed_File = r"C:\program files\arcgis\pro\Resources\ArcToolbox\Templates\CAD\template2d_US_Feet.dgn"
)
print(f"Polylines and/or Polygons Exported: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Started: 2024-08-02 13:35:08
Polylines and/or Polygons Exported: 2024-08-02 13:35:12
